In [ ]:
import json
import numpy as np
import pandas as pd
#import geopandas as gpd
import json
import re
import os

from tqdm.notebook import tqdm
from opencage.geocoder import OpenCageGeocode

# ============================================
# clean tranaction price df
# ============================================
def clean_transaction_price():
    # load raw data:
    df = pd.read_csv('../../data/raw/transaction_price.csv', sep = ',').drop_duplicates()
    df = df.loc[~df.district.isnull()]
    print(f'Orignal df transaction price: {df.shape}')
    
    # clean district name:
    df['district'] = df['district'].map(lambda x: re.sub('/', '-',str(x)) )
    df['district'] = df['district'].map(lambda x: re.sub(',', '-',str(x)) )
    df['district'] = df['district'].map(lambda x: re.sub('\s', '',str(x)) )
    df['district'] = df['district'].str.title()
    
    # number of rooms
    df['apartment_type'] = df['apartment_type'].str.lower()
    df['number_of_room'] = df['apartment_type'].map(lambda x: x[:1] if not pd.isna(x) else None)
    df['number_of_room'] = pd.to_numeric(df['number_of_room'] , errors='coerce')
    
    df['number_of_room_cat'] = df['number_of_room'].map(lambda x: str(round(x)) if not pd.isna(x) else None)
    df['number_of_room_cat'] = df['number_of_room'].map(lambda x: '3+' if float(x) >=3 else x)
    df['number_of_room_cat'] = df['number_of_room_cat'].astype('str')
    
    # property type
    df['property_type']= df['property_type'].replace({'kt':'apartment', 'rt':'row house', \
                                                  'ok':'single-family house'})
    df['property_type_main'] = df['property_type'].map(lambda x: 'house' if x!='apartment' else x)
    
    # clean plot column
    df['plot'] = df['plot'].str.replace('\W+', '')
    # translate condition to English
    condition = { 'tyyd.' : 'Satisfying',
                 'huono': 'Poor',
                 'hyvä': 'Good',
                 np.nan: 'Not classified',  
    }
    df.condition = df.condition.replace(condition)
    

    return df
    
def geocode_address(df):
    # One need to register API key with opencage service
    key = os.getenv('opencage_api')
    geocoder = OpenCageGeocode(key)
    postcode_dict = {}
    district_geocode = df['district'].unique() # find list of unique postcode
    district_geocode = [s + ', Finland'  for s in district_geocode]

    for i in range(0, len(district_geocode)):
        result= geocoder.geocode(district_geocode[i])
        if len(result) > 1:            
            postcode_dict[district_geocode[i]]= result[1]['components']['postcode']
        else:       
            postcode_dict[district_geocode[i]]= pd.DataFrame(result[0])['formatted'].iloc[0]

            
    return postcode_dict    

def merge_district(postcode_dict, df):
    df_postcode = pd.DataFrame(postcode_dict, index=[0]).T.reset_index()      
    df_postcode.columns= ['district', 'postcode']
    
    df_postcode['district'] = df_postcode['district'].str.replace(', Finland', '')
    df_postcode['postcode'] = df_postcode['postcode'].map(lambda s: re.findall('\d{5}', s) )
    df_postcode['postcode'] = df_postcode['postcode'].map(lambda x: x[0] if len(x) >0 else None)
                                                         
    df = df.merge(df_postcode, on='district')
            
    return  df

def geocode_address(df):
    key = os.getenv('opencage_api')
    key = '889e64fd8634412baa255b24c0e06f3f'
    geocoder = OpenCageGeocode(key)

    postcode_dict = {}
    postcode_null = []
    
    # extract municipality
    df['municipality'] = df.municipaliy.map(lambda x: re.sub(r'(.*)(c=)(\w+)(&.*)', r"\3", x) )
    print(df.head())
    df['district'] = df['district'] + ', ' + df['municipality']
    
    district_geocode = df['district'].unique() # find list of unique postcode
    district_geocode = [s + ', Finland'  for s in district_geocode]

    for i in tqdm(range(0, len(district_geocode))):
        result= geocoder.geocode(district_geocode[i])
        if len(result) > 1:
            try:
                postcode_dict[district_geocode[i]]= result[1]['components']['postcode']
            except :
                print(district_geocode[i])
                postcode_null.append(district_geocode[i])
        else:       
            postcode_dict[district_geocode[i]]= pd.DataFrame(result[0])['formatted'].iloc[0]
      
    return postcode_dict, postcode_null    


def merge_district(postcode_dict, df):  
    '''Merge 2 gecode dataframe so we have postcodes in transaction price'''
    df_postcode = pd.DataFrame(postcode_dict, index=[0]).T.reset_index()   
    df_postcode.columns= ['district', 'postcode']
    
    df_postcode['district'] = df_postcode['district'].str.replace(', Finland', '')
    df_postcode['postcode'] = df_postcode['postcode'].map(lambda s: re.findall('\d{5}', s) )
    df_postcode['postcode'] = df_postcode['postcode'].map(lambda x: x[0] if len(x) >0 else None)
                                                          
    df= df.merge(df_postcode, on='district')
    print('Geocode postcode for df transaction price: done')
            
    return  df
 
def process_transaction_df():
    df = clean_transaction_price()
    postcode_geo, postcode_null = geocode_address(df) 
    df = merge_district(postcode_geo, df)
    df['postcode'] = df['postcode'].astype(str).str.pad(5, "left", "0")
    df.to_csv('../../data/processed/transaction_price.csv', index=False)
    print('==== Save df transaction price: done ====')
       
    return df


# ============================================
# clean asking price df
# ============================================

def clean_df_listing():
    df = pd.read_csv("../data/raw/Espoo_Vantaa.csv").drop_duplicates()
    df['postcode'] = df['postcode'].astype(str).str.pad(5, "left", "0")
    df['asking_price_m2'] = df['price'] / df['floor_area']

    # clean municipality
    df["municipality"]=df["municipality"].astype('category')
    # number_of_room
    df['number_of_room'] = df['house_info'].str.extract(pat =  "(\d\s*m*h*\s?\+*)")
    df['number_of_room'] = df['number_of_room'].str.extract('(\d+)')
    df['number_of_room'] = df['number_of_room'].map(lambda x: float(x) if not pd.isna(x) else x)

    df['number_of_room_cat'] = df['number_of_room'].map(lambda x: str(round(x)) if not pd.isna(x) else None)
    df['number_of_room_cat'] = df['number_of_room'].map(lambda x: '3+' if float(x) >=3 else x)
    df['number_of_room_cat'] = df['number_of_room_cat'].astype('str')
    
    # maitenance cost per sqm
    df['maintenance_cost_m2'] = df['Unit maintenance cost'] / df['floor_area']
    df['maintenance_cost_m2'] = df['maintenance_cost_m2'].map(lambda x: 0 if pd.isna(x) else x)
    
    # filter potential wrong values
    df = df[ ~((df['number_of_room'] <1 )| (df['number_of_room'].isnull() )) ]
    df = df[(df['asking_price_m2']>1)  &  (df['asking_price_m2']<20000)] # local knowledge: rarely there is \
                                                           # property sold with >20k/m2
    # divide propery into 2 groups: house and apartment:
    df['property_type_main'] = df['property_type'].map(lambda x: 'house' if x!='apartment' else x)
    
    df.to_csv('../data/processed/asking_price.csv', index=False)
    print('==== Save df asking price: done ====')
    
    return df

# ============================================
# clean rent df
# ============================================

def load_rent():
    # load file
    df = pd.read_json('../data/raw/rent.jl', lines=True,orient='records').drop_duplicates()
    
    # clean columns
    df['postcode'] = df['postcode'].map(lambda x: re.sub('\D+', '', x))
    df['number_of_room'] = df['apartment_type'].map(lambda x: re.sub('h', '', x))
    df = df.loc[df['number_of_room'].isin(['1', '2', '3+'])]
    
    # change dtype of rent columns
    df[['nonsub_old', 'nonsub_new']] = df[['nonsub_old', 'nonsub_new']].apply(pd.to_numeric, errors='coerce')
    
    # drop unneeded columns:
    df.drop(['apartment_type', 'ARA_rental'], axis=1, inplace=True)

    df.to_csv('../data/processed/rent.csv', index=False)
    print('==== Save df rent: done ====')
    
    return df


# ============================================
# merge df_rent, df_price and df_listing
# ============================================

def caculate_yield(x):
    if x['new']:
        x['net_yield'] = 12*(x['nonsub_new'] - x['maintenance_cost_m2']) / x['asking_price_m2']
    else:
        x['net_yield']= 12*(x['nonsub_old'] - x['maintenance_cost_m2']) / x['asking_price_m2']
    
    x['net_yield']= np.round(x['net_yield'] * 100, 2)
    
    return x['net_yield']

def merge_df(df_rent, df_listing, df_price):
    # correct format of postcode:
    for df in [df_rent, df_listing, df_price]:
        df['postcode'] = df['postcode'].map(lambda x: str(np.int(x)) if not pd.isna(x) else x )
        df['postcode'] = df['postcode'].astype(str).str.pad(width=5, side='left', fillchar='0')
        if 'number_of_room_cat' in df.columns:
            df['number_of_room_cat'] = df['number_of_room_cat'].replace({'1.0': '1' , '2.0':'2'})
            
    # merge price and listing

    df_price = df_price.groupby(['postcode', 'number_of_room_cat','property_type_main'])\
                ['price_m2'].mean().reset_index()
    df_listing_agg = df_listing.groupby(['postcode', 'number_of_room_cat','property_type_main'])\
                ['asking_price_m2'].mean().reset_index()
    
    df_listing_price = df_listing_agg.merge(df_price[['postcode', 'number_of_room_cat','price_m2']], on=['postcode', 'number_of_room_cat'],\
                      how='left').drop_duplicates()
    
    # merge df listing and df rent
    df_yield = df_listing.merge(df_rent, left_on = ['postcode', 'number_of_room_cat'], \
                            right_on = ['postcode', 'number_of_room'], how='left')
   
    print(df_yield['maintenance_cost_m2'].isnull().mean())
    df_yield['new'] = df_yield['build_year'].map(lambda x: True if x >= 2015 else False)
    df_yield['net_yield'] = df_yield.apply(caculate_yield, axis=1)
    
    df_yield.to_csv('../data/processed/df_yield.csv', index=False)
    df_listing_price.to_csv('../data/processed/df_listing_price.csv', index=False)
   
    print('==== Save df yield and price: done ====')
    
    
    return df_listing_price, df_yield
        

if __name__ == "__main__":
    print('Start')
    # df_rent = load_rent()
    df_price = process_transaction_df()
    # df_listing = clean_df_listing()
    # df_listing_price, df_yield =  merge_df(df_rent, df_listing, df_price)


Start
Orignal df transaction price: (7312, 14)
     district      apartment_type property_type floor_area     price  \
1  Leppävaara  1h, kk, kph, pa...     apartment      25,00  146000.0   
2  Haukilahti         1h+kk+kph+p     apartment      25,00  170000.0   
3     Tapiola  1h, kk, kph, ra...     apartment      28,00  210000.0   
4      Soukka  1h, k, kph, las...     apartment      31,00  145000.0   
5     Tapiola  1h, k, kph, vh ...     apartment      41,00  216356.0   

   price_m2  build_year floor elevator   condition plot energy_class  \
1    5840.0      1963.0   2/4       ei  Satisfying  oma            E   
2    6800.0      1965.0   4/4       on        Good  oma            D   
3    7500.0      1964.0   5/5       on        Good  oma            E   
4    4677.0      1974.0   8/9       on        Good  oma            D   
5    5277.0      1958.0   2/4       ei  Satisfying  oma            E   

  date_scrape                                        municipaliy  \
1  20/02/2021  http

<ipython-input-48-c4d318ba1a91>:42: FutureWarning: The default value of regex will change from True to False in a future version.
  df['plot'] = df['plot'].str.replace('\W+', '')


Perkkaa, Espoo, Finland
Kivenlahti, Espoo, Finland
Espoonlahti, Espoo, Finland
Tontunmäki, Espoo, Finland
Espoonlahti-Laurinlahti, Espoo, Finland
Lähderanta-Laaksolahti, Espoo, Finland
Nuottaniemi-Matinkylä, Espoo, Finland
Lippajärvi, Espoo, Finland
Sepänkylä, Espoo, Finland
Lähderanta, Espoo, Finland
Soukanniemi, Espoo, Finland
Vallikallio, Espoo, Finland
Meri-Kivenlahti, Espoo, Finland
Bemböle-Lommila-Fallåker, Espoo, Finland
Ylä-Soukka, Espoo, Finland
Koivuhovi-Sepänkylä, Espoo, Finland
Olari-Matinkylä, Espoo, Finland
Suurpelto, Espoo, Finland
Kalajärvi, Espoo, Finland
Hyljelahti-Iivisniemi, Espoo, Finland
Tillinmäki, Espoo, Finland
Kirkkojärvi, Espoo, Finland
Piispankylä, Espoo, Finland
Iirislahti, Espoo, Finland
Hannusjärvi, Espoo, Finland
Lystimäki, Espoo, Finland
Nuottaniemi, Espoo, Finland
Hannus, Espoo, Finland
Koivuhovi, Espoo, Finland
Pohjois-Tapiola-Tapiola, Espoo, Finland
Sokinvuori, Espoo, Finland
Piispanpiha, Espoo, Finland
Tiistilä, Espoo, Finland
Bemböle-Fallåker, Espo

In [8]:
df = pd.read_csv('../../data/processed/transaction_price.csv')

<ipython-input-20-94b743d3a7d6>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['plot'] = df['plot'].str.replace('\W+', '')


In [21]:
df

,district,apartment_type,property_type,floor_area,price,price_m2,build_year,floor,elevator,condition,plot,energy_class,date_scrape,municipaliy,number_of_room,number_of_room_cat,property_type_main,postcode
0,Leppävaara,"1h, kk, kph, pa...",apartment,"25,00",146000.0,5840.0,1963.0,2/4,ei,Satisfying,oma,E,20/02/2021,https://asuntojen.hintatiedot.fi/haku/?c=Espoo...,1.0,1.0,apartment,2600.0
1,Leppävaara,"1h, kk, ph, s, ...",apartment,"28,50",158900.0,5575.0,1985.0,1/6,on,Good,oma,F,20/02/2021,https://asuntojen.hintatiedot.fi/haku/?c=Espoo...,1.0,1.0,apartment,2600.0
2,Leppävaara,"3h, k, s",apartment,"68,50",273300.0,3990.0,1989.0,4/6,on,Good,oma,NaN,20/02/2021,https://asuntojen.hintatiedot.fi/haku/?c=Espoo...,3.0,3+,apartment,2600.0
3,Leppävaara,"3h, k, s, vh, l...",apartment,"77,00",215000.0,2792.0,1987.0,1/4,on,Satisfying,oma,F,20/02/2021,https://asuntojen.hintatiedot.fi/haku/?c=Espoo...,3.0,3+,apartment,2600.0
4,Leppävaara,"3h, k, s, lasit...",apartment,"65,00",302000.0,4646.0,2018.0,2/5,on,Good,,C,20/02/2021,https://asuntojen.hintatiedot.fi/haku/?c=Espoo...,3.0,3+,apartment,2600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,Vuosaari-Rastila,4h+k+s,apartment,"78,00",203000.0,2603.0,1990.0,3/3,on,Good,vuokra,E,20/02/2021,https://asuntojen.hintatiedot.fi/haku/?c=Helsi...,4.0,3+,apartment,980.0
5763,Puumyllypuro-Vartiokylä,6h+k+2xkph+s,row house,"139,00",460000.0,3309.0,2014.0,2/2,ei,Good,vuokra,NaN,20/02/2021,https://asuntojen.hintatiedot.fi/haku/?c=Helsi...,6.0,3+,house,NaN
5764,Malminkartano-Kaarela,5h+k+2wc+sauna+...,apartment,"120,50",245000.0,2033.0,1985.0,4/4,ei,Satisfying,vuokra,F,20/02/2021,https://asuntojen.hintatiedot.fi/haku/?c=Helsi...,5.0,3+,apartment,410.0
5765,Aurinkolahti-Vuosaari,"4h, k, kph, s, ...",apartment,"94,50",455000.0,4815.0,2002.0,4/8,on,Satisfying,oma,F,20/02/2021,https://asuntojen.hintatiedot.fi/haku/?c=Helsi...,4.0,3+,apartment,990.0


In [35]:
df.municipaliy.iloc[0]



In [36]:
df

,district,apartment_type,property_type,floor_area,price,price_m2,build_year,floor,elevator,condition,plot,energy_class,date_scrape,municipaliy,number_of_room,number_of_room_cat,property_type_main,postcode
0,Leppävaara,"1h, kk, kph, pa...",apartment,"25,00",146000.0,5840.0,1963.0,2/4,ei,Satisfying,oma,E,20/02/2021,Espoo,1.0,1.0,apartment,2600.0
1,Leppävaara,"1h, kk, ph, s, ...",apartment,"28,50",158900.0,5575.0,1985.0,1/6,on,Good,oma,F,20/02/2021,Espoo,1.0,1.0,apartment,2600.0
2,Leppävaara,"3h, k, s",apartment,"68,50",273300.0,3990.0,1989.0,4/6,on,Good,oma,NaN,20/02/2021,Espoo,3.0,3+,apartment,2600.0
3,Leppävaara,"3h, k, s, vh, l...",apartment,"77,00",215000.0,2792.0,1987.0,1/4,on,Satisfying,oma,F,20/02/2021,Espoo,3.0,3+,apartment,2600.0
4,Leppävaara,"3h, k, s, lasit...",apartment,"65,00",302000.0,4646.0,2018.0,2/5,on,Good,,C,20/02/2021,Espoo,3.0,3+,apartment,2600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,Vuosaari-Rastila,4h+k+s,apartment,"78,00",203000.0,2603.0,1990.0,3/3,on,Good,vuokra,E,20/02/2021,Helsinki,4.0,3+,apartment,980.0
5763,Puumyllypuro-Vartiokylä,6h+k+2xkph+s,row house,"139,00",460000.0,3309.0,2014.0,2/2,ei,Good,vuokra,NaN,20/02/2021,Helsinki,6.0,3+,house,NaN
5764,Malminkartano-Kaarela,5h+k+2wc+sauna+...,apartment,"120,50",245000.0,2033.0,1985.0,4/4,ei,Satisfying,vuokra,F,20/02/2021,Helsinki,5.0,3+,apartment,410.0
5765,Aurinkolahti-Vuosaari,"4h, k, kph, s, ...",apartment,"94,50",455000.0,4815.0,2002.0,4/8,on,Satisfying,oma,F,20/02/2021,Helsinki,4.0,3+,apartment,990.0
